In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as T
from PIL import Image
import os
import shutil
from matplotlib import pyplot as plt


In [2]:
def get_unet(in_channels=3, out_channels=1):
    def conv_block(in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    class UNet(nn.Module):
        def __init__(self):
            super(UNet, self).__init__()
            self.enc1 = conv_block(in_channels, 64)
            self.enc2 = conv_block(64, 128)
            self.enc3 = conv_block(128, 256)
            self.enc4 = conv_block(256, 512)
            self.pool = nn.MaxPool2d(2)
            self.middle = conv_block(512, 1024)

            self.up4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
            self.dec4 = conv_block(1024, 512)
            self.up3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
            self.dec3 = conv_block(512, 256)
            self.up2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
            self.dec2 = conv_block(256, 128)
            self.up1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
            self.dec1 = conv_block(128, 64)

            self.final = nn.Conv2d(64, out_channels, kernel_size=1)

        def forward(self, x):
            e1 = self.enc1(x)
            e2 = self.enc2(self.pool(e1))
            e3 = self.enc3(self.pool(e2))
            e4 = self.enc4(self.pool(e3))
            m = self.middle(self.pool(e4))

            d4 = self.dec4(torch.cat([self.up4(m), e4], dim=1))
            d3 = self.dec3(torch.cat([self.up3(d4), e3], dim=1))
            d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
            d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))

            return self.final(d1)

    return UNet()

In [3]:
class EarlyStopping:
    def __init__(self, patience=20, delta=0, mode='max'):  # <-- Customize parameters here
        self.patience = patience
        self.delta = delta
        self.mode = mode
        self.best_score = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif (self.mode == 'max' and score < self.best_score + self.delta) or \
             (self.mode == 'min' and score > self.best_score - self.delta):
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

In [4]:
def calculate_iou(pred, target):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = ((pred + target) > 0).float().sum()
    if union == 0:
        return 0.0
    return (intersection / union).item()

In [5]:
class RoadCrackDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_names = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_names[idx])
        mask_path = os.path.join(self.mask_dir, self.image_names[idx])
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [6]:
def train_model(model, train_loader, val_loader, optimizer, device, num_epochs=100):
    early_stopping = EarlyStopping(patience=20, mode='max')  # <-- Change patience/mode here
    best_iou = 0.0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = F.binary_cross_entropy_with_logits(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        model.eval()
        iou_scores = []
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                iou = calculate_iou(outputs, masks)
                iou_scores.append(iou)

        avg_iou = sum(iou_scores) / len(iou_scores)
        print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val IOU: {avg_iou:.4f}")

        early_stopping(avg_iou)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

In [7]:
def test_single_image(model, transform, device):
    image = Image.open("/kaggle/input/udtiri-crack/UDTIRI-Crack/UDTIRI-Crack Detection/test/image/1_008.png").convert("RGB")
    tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(tensor)
        mask = torch.sigmoid(output).squeeze().cpu().numpy()
        mask = (mask > 0.5).astype('uint8') * 255

    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title("Input Image")
    plt.subplot(1,2,2)
    plt.imshow(mask, cmap='gray')
    plt.title("Predicted Mask")
    plt.show()

In [8]:
def setup_model_and_device():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = get_unet()

    # Use multiple GPUs if available
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # <-- Change LR here if needed
    return model, optimizer, device

In [9]:
def main(image_dir, mask_dir, val_img_dir, val_mask_dir,
         batch_size=4, lr=1e-4, num_epochs=100, patience=20):
    transform = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor()
    ])

    train_dataset = RoadCrackDataset(image_dir, mask_dir, transform=transform)
    val_dataset = RoadCrackDataset(val_img_dir, val_mask_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model, optimizer, device = setup_model_and_device()
    train_model(model, train_loader, val_loader, optimizer, device, num_epochs=num_epochs)

In [10]:
if __name__ == "__main__":
    src_image = "/kaggle/input/udtiri-crack/UDTIRI-Crack/UDTIRI-Crack Detection/train/image"
    src_mask = "/kaggle/input/udtiri-crack/UDTIRI-Crack/UDTIRI-Crack Detection/train/label"
    val_image = "/kaggle/input/udtiri-crack/UDTIRI-Crack/UDTIRI-Crack Detection/val/image"
    val_mask = "/kaggle/input/udtiri-crack/UDTIRI-Crack/UDTIRI-Crack Detection/val/label"

    image_dir = "/kaggle/working/images"
    mask_dir = "/kaggle/working/masks"
    val_img_dir = "/kaggle/working/val_images"
    val_mask_dir = "/kaggle/working/val_masks"

    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_mask_dir, exist_ok=True)

    for f in os.listdir(src_image):
        shutil.copy(os.path.join(src_image, f), os.path.join(image_dir, f))
    for f in os.listdir(src_mask):
        shutil.copy(os.path.join(src_mask, f), os.path.join(mask_dir, f))
    for f in os.listdir(val_image):
        shutil.copy(os.path.join(val_image, f), os.path.join(val_img_dir, f))
    for f in os.listdir(val_mask):
        shutil.copy(os.path.join(val_mask, f), os.path.join(val_mask_dir, f))

    main(
        image_dir=image_dir,
        mask_dir=mask_dir,
        val_img_dir=val_img_dir,
        val_mask_dir=val_mask_dir,
        batch_size=4,
        lr=1e-4,
        num_epochs=100,
        patience=20
    )

Using 2 GPUs
Epoch 1, Train Loss: 0.1418, Val IOU: 0.1971
Epoch 2, Train Loss: 0.0650, Val IOU: 0.2399
Epoch 3, Train Loss: 0.0581, Val IOU: 0.2180
Epoch 4, Train Loss: 0.0565, Val IOU: 0.2404
Epoch 5, Train Loss: 0.0535, Val IOU: 0.2379
Epoch 6, Train Loss: 0.0519, Val IOU: 0.2506
Epoch 7, Train Loss: 0.0489, Val IOU: 0.2751
Epoch 8, Train Loss: 0.0467, Val IOU: 0.2596
Epoch 9, Train Loss: 0.0456, Val IOU: 0.3006
Epoch 10, Train Loss: 0.0431, Val IOU: 0.3211
Epoch 11, Train Loss: 0.0433, Val IOU: 0.2935
Epoch 12, Train Loss: 0.0409, Val IOU: 0.3135
Epoch 13, Train Loss: 0.0403, Val IOU: 0.3355
Epoch 14, Train Loss: 0.0389, Val IOU: 0.2829
Epoch 15, Train Loss: 0.0377, Val IOU: 0.2992
Epoch 16, Train Loss: 0.0371, Val IOU: 0.2927
Epoch 17, Train Loss: 0.0359, Val IOU: 0.3033
Epoch 18, Train Loss: 0.0354, Val IOU: 0.2995
Epoch 19, Train Loss: 0.0340, Val IOU: 0.3344
Epoch 20, Train Loss: 0.0333, Val IOU: 0.3411
Epoch 21, Train Loss: 0.0322, Val IOU: 0.3132
Epoch 22, Train Loss: 0.0313, 